In [1]:
!pip install -q openai faiss-cpu sentence-transformers





In [2]:
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
from google.colab import userdata
from openai import OpenAI


print("✅ Imports successful")



✅ Imports successful


In [4]:
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

if not os.environ.get("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in Colab Secrets")

client = OpenAI()
print("✅ OpenAI client initialized")



✅ OpenAI client initialized


In [5]:
qa_documents = [
    "Always include positive, negative, and edge test cases.",
    "Validate empty inputs and invalid formats.",
    "Verify proper error messages for failures.",
    "Login with valid credentials should succeed.",
    "Login with invalid password should display an error message.",
    "Login with empty email or password should show validation error.",
    "Password reset with registered email should send reset link.",
    "Password reset with invalid email should show error message."
]


In [6]:
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

doc_embeddings = embedding_model.encode(qa_documents)

dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(doc_embeddings))

print("✅ FAISS vector store created")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ FAISS vector store created


In [7]:
def retrieve_context(query, k=3):
    query_embedding = embedding_model.encode([query])
    distances, indices = index.search(np.array(query_embedding), k)
    return "\n".join([qa_documents[i] for i in indices[0]])


In [11]:
def generate_test_cases(requirement):
    context = retrieve_context(requirement)

    prompt = f"""
You are a professional QA engineer.

Context:
{context}

Software Requirement:
{requirement}

Generate EXACTLY 6 test cases in VALID JSON.

Rules:
- Output ONLY JSON (array)
- Do NOT use "..."
- Do NOT include explanations
- Each object must contain:
  id, scenario, steps, expected_result
"""

    response = client.chat.completions.create(
        model="gpt-5-nano",
        messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content


In [12]:
requirement = """
Feature: User Login
User enters email and password
System validates credentials
Redirect to dashboard on success
Show error on failure
"""


In [14]:
print("=== AI-Driven Automated Test Case Generator ===")
print("Enter the requirement below.")
print("Type 'exit' to stop.\n")

while True:
    user_input = input("Enter software requirement:\n")

    if user_input.lower().strip() == "exit":
        print("Exiting test case generator. Goodbye!")
        break

    print("\nGenerating test cases...\n")

    output = generate_test_cases(user_input)
    print(output)
    print("\n" + "="*60 + "\n")


=== AI-Driven Automated Test Case Generator ===
Enter the software requirement below.
Type 'exit' to stop.

Enter software requirement:
User should be able to reset password using email OTP

Generating test cases...

[
  {
    "id": "TC-PR-001",
    "scenario": "Password reset with invalid email should show error message.",
    "steps": [
      "Open the application login page.",
      "Click 'Forgot Password' link.",
      "Enter an invalid email format, e.g., 'user@domain' or 'invalid-email', in the email field.",
      "Click Submit/Reset Password."
    ],
    "expected_result": "An error message indicating invalid email format is displayed and password reset is not initiated."
  },
  {
    "id": "TC-PR-002",
    "scenario": "Password reset with registered email should send reset link.",
    "steps": [
      "Open the application login page.",
      "Click 'Forgot Password' link.",
      "Enter a registered email address, e.g., 'registered@example.com', in the email field.",
      "